In [19]:
import numpy as np
from sympy import *

In [20]:
m = 1550     # kg
theta = 2800 # kg m^2
l_v = 1.344  # m
l_h = 1.456  # m
c_v = 75000  # N / rad
c_h = 150000 # N / rad
i_S = 16

In [21]:
# derived parameters
l = l_v + l_h

In [22]:
from IPython.display import Image

In [23]:
Image(url="https://upload.wikimedia.org/wikipedia/commons/2/24/EinspurKinematik.png")

In [69]:
# in the single track model, v is assumed to be constant
# I want to be able to simulate different values of v
# so the velocity is kept symbolic
v = symbols("v")


In [74]:
# dx = A*x + B*u
# where x=[beta, psi_dot]
#       u=[delta / i_s]

beta, psi_dot = symbols("beta psi_dot")
delta = symbols("delta")

A = Matrix([[-(c_v + c_h)/m*v, (m*v**2 - (c_h*l_h - c_v*l_v))/(m*v**2)],
           [- (c_h*l_h - c_v*l_v)/theta, -(c_h*l_h**2 + c_v*l_v**2)/(theta*v)]])

B = Matrix([[-c_v / (m*v)], [c_v*l_v/theta]])

In [87]:
def dstate(state, u):
    return A @ state + B@u

In [103]:
T = 0.01

In [104]:
state = Matrix([beta, psi_dot])
u = Matrix([delta / i_S])

# runge kutta
k1 = dstate(state, u)
k2 = dstate(state + T/2*k1, u)
k3 = dstate(state + T/2*k2, u)
k4 = dstate(state + T*k3, u)

rk = state + T/6 * (k1 + 2*k2 + 2*k3 + k4)

In [106]:
beta_val = 0
psi_dot_val = 0
delta_val = 0.05
v_val = 10

substitutions = {beta: beta_val, psi_dot : psi_dot_val, delta: delta_val, v:v_val}

rk.subs(substitutions).evalf()

Matrix([
[ 0.0149262386695396],
[0.00147941011090546]])

In [107]:
rk

Matrix([
[-0.241935483870968*beta*v + beta - 0.030241935483871*delta/v + 1.0752688172043e-6*psi_dot*(1550*v**2 - 117600.0)/v**2 - 0.241935483870968*v*(beta - 0.030241935483871*delta/v - 1.45161290322581*v*(beta - 0.0151209677419355*delta/v - 0.725806451612903*v*(-0.725806451612903*beta*v + beta - 0.0151209677419355*delta/v + 3.2258064516129e-6*psi_dot*(1550*v**2 - 117600.0)/v**2) + 3.2258064516129e-6*(1550*v**2 - 117600.0)*(-0.21*beta + 0.01125*delta + psi_dot - 0.80976*psi_dot/v)/v**2) + 6.45161290322581e-6*(1550*v**2 - 117600.0)*(0.15241935483871*beta*v - 0.21*beta + 0.01125*delta + 0.00317540322580645*delta/v + psi_dot - 6.7741935483871e-7*psi_dot*(1550*v**2 - 117600.0)/v**2 - 0.80976*(-0.21*beta + 0.01125*delta + psi_dot - 0.80976*psi_dot/v)/v)/v**2) - 0.483870967741935*v*(beta - 0.0151209677419355*delta/v - 0.725806451612903*v*(-0.725806451612903*beta*v + beta - 0.0151209677419355*delta/v + 3.2258064516129e-6*psi_dot*(1550*v**2 - 117600.0)/v**2) + 3.2258064516129e-6*(1550*v**2 - 1